In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
file_path = "/content/drive/MyDrive/output_file.csv"
df = pd.read_csv(file_path)
df.head()

,Player,Runs,Mins,BF,4s,6s,SR,Inns,Unnamed: 8,Opposition,Ground,Start Date,Unnamed: 12
0,G Boycott (ENG),8,34,37,0,0,21.62,1,NaN,v Australia,Melbourne,5 Jan 1971,NaN
1,JH Edrich (ENG),82,150,119,4,0,68.90,1,NaN,v Australia,Melbourne,5 Jan 1971,NaN
2,KWR Fletcher (ENG),24,60,47,1,0,51.06,1,NaN,v Australia,Melbourne,5 Jan 1971,NaN
3,BL D'Oliveira (ENG),17,20,16,1,0,106.25,1,NaN,v Australia,Melbourne,5 Jan 1971,NaN
4,JH Hampshire (ENG),10,13,13,0,0,76.92,1,NaN,v Australia,Melbourne,5 Jan 1971,NaN


In [4]:
print(df.columns)

Index(['Player', 'Runs', 'Mins', 'BF', '4s', '6s', 'SR', 'Inns', 'Unnamed: 8',
       'Opposition', 'Ground', 'Start Date', 'Unnamed: 12'],
      dtype='object')


In [5]:
columns_to_remove = ['Unnamed: 8', 'Unnamed: 12']
df = df.drop(columns=columns_to_remove, axis=1)

In [6]:
df.head()

,Player,Runs,Mins,BF,4s,6s,SR,Inns,Opposition,Ground,Start Date
0,G Boycott (ENG),8,34,37,0,0,21.62,1,v Australia,Melbourne,5 Jan 1971
1,JH Edrich (ENG),82,150,119,4,0,68.90,1,v Australia,Melbourne,5 Jan 1971
2,KWR Fletcher (ENG),24,60,47,1,0,51.06,1,v Australia,Melbourne,5 Jan 1971
3,BL D'Oliveira (ENG),17,20,16,1,0,106.25,1,v Australia,Melbourne,5 Jan 1971
4,JH Hampshire (ENG),10,13,13,0,0,76.92,1,v Australia,Melbourne,5 Jan 1971


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103948 entries, 0 to 103947
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Player      103948 non-null  object
 1   Runs        103948 non-null  object
 2   Mins        103948 non-null  object
 3   BF          103948 non-null  object
 4   4s          103948 non-null  object
 5   6s          103948 non-null  object
 6   SR          103948 non-null  object
 7   Inns        103948 non-null  object
 8   Opposition  103948 non-null  object
 9   Ground      103948 non-null  object
 10  Start Date  103948 non-null  object
dtypes: object(11)
memory usage: 8.7+ MB


In [8]:
df.isna().sum()

Player        0
Runs          0
Mins          0
BF            0
4s            0
6s            0
SR            0
Inns          0
Opposition    0
Ground        0
Start Date    0
dtype: int64

## Data Cleaning

In [9]:
#Runs column has entries of players who did not bat in that innings

values_to_remove = ['sub', 'DNB', 'TDNB', 'absent']
df = df.query('Runs not in @values_to_remove')
df.reset_index(drop=True, inplace=True)
df['Runs'] = df['Runs'].str.replace('*', '')

<ipython-input-9-30774443f615>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Runs'] = df['Runs'].str.replace('*', '')
<ipython-input-9-30774443f615>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Runs'] = df['Runs'].str.replace('*', '')


In [10]:
#Minutes batted has several missing values, replacing them with the average
df['Mins'] = pd.to_numeric(df['Mins'].replace('-', np.nan))
average_mins = df['Mins'].mean()
df['Mins'].fillna(average_mins, inplace=True)

In [11]:
#4s and 6s had  "-" instead of 0
df['4s'] = df['4s'].replace('-', 0)
df['6s'] = df['6s'].replace('-', 0)

#players with zero runs were given strike rate of "-", converting them to 0
df['SR'] = df['SR'].replace('-', 0)

In [12]:
df['4s'] = pd.to_numeric(df['4s']).astype(int)
df['6s'] = pd.to_numeric(df['6s']).astype(int)
df['BF'] = pd.to_numeric(df['BF']).astype(int)
df['Mins'] = pd.to_numeric(df['Mins']).astype(int)
df['Runs'] = pd.to_numeric(df['Runs']).astype(int)
df['SR'] = pd.to_numeric(df['SR']).astype(float)
df['Inns'] = pd.to_numeric(df['Inns']).astype(int)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82658 entries, 0 to 82657
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Player      82658 non-null  object 
 1   Runs        82658 non-null  int64  
 2   Mins        82658 non-null  int64  
 3   BF          82658 non-null  int64  
 4   4s          82658 non-null  int64  
 5   6s          82658 non-null  int64  
 6   SR          82658 non-null  float64
 7   Inns        82658 non-null  int64  
 8   Opposition  82658 non-null  object 
 9   Ground      82658 non-null  object 
 10  Start Date  82658 non-null  object 
dtypes: float64(1), int64(6), object(4)
memory usage: 6.9+ MB


In [14]:
#creating a new column "teams"
df['Team'] = df['Player'].str.extract(r'\((\w+)\)')

# Remove the extracted team information from the "Player" column
df['Player'] = df['Player'].str.replace(r'\s*\(\w+\)', '')

# Remove "v" and the following space from the "Opposition" column
df['Opposition'] = df['Opposition'].str.replace(r'^v\s+', '')


<ipython-input-14-27a125131815>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Player'] = df['Player'].str.replace(r'\s*\(\w+\)', '')
<ipython-input-14-27a125131815>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Opposition'] = df['Opposition'].str.replace(r'^v\s+', '')


In [23]:
df['Start Date'] = pd.to_datetime(df['Start Date'])

# Extract year and month
df['Year'] = df['Start Date'].dt.year
df['Month'] = df['Start Date'].dt.month

#creating a new column decade
df['Decade'] = (df['Year']// 10) * 10

In [16]:
#calcluating the runs scored by running between the wickets and storing in a new column
df['runs_by_running'] = df['Runs'] - (4 * df['4s'] + 6 * df['6s'])
df['perc_runs_by_running'] = np.where(df['Runs'] == 0, 0, round((df['runs_by_running'] / df['Runs']) * 100, 2))


In [19]:
#Adding a new column called Match_number to assign a unique number to each match
counter = 0
match_numbers = {}

def assign_match_number(row):
    global counter
    key = ( row['Ground'], row['Start Date'])
    if key not in match_numbers:
        counter += 1
        match_numbers[key] = counter
    return match_numbers[key]

df['Match_Number'] = df.apply(assign_match_number, axis=1)

In [30]:
df.tail()

,Player,Runs,Mins,BF,4s,6s,SR,Inns,Opposition,Ground,Start Date,Team,Year,Month,runs_by_running,perc_runs_by_running,Match_Number,Decade
82653,D Madushanka,3,1,2,0,0,150.00,1,Zimbabwe,Colombo (RPS),2024-01-06,SL,2024,1,3,100.0,4693,2020
82654,T Kaitano,1,20,16,0,0,6.25,2,Sri Lanka,Colombo (RPS),2024-01-06,ZIM,2024,1,1,100.0,4693,2020
82655,TS Kamunhukamwe,0,10,1,0,0,0.00,2,Sri Lanka,Colombo (RPS),2024-01-06,ZIM,2024,1,0,0.0,4693,2020
82656,CR Ervine,0,3,4,0,0,0.00,2,Sri Lanka,Colombo (RPS),2024-01-06,ZIM,2024,1,0,0.0,4693,2020
82657,M Shumba,2,5,3,0,0,66.66,2,Sri Lanka,Colombo (RPS),2024-01-06,ZIM,2024,1,2,100.0,4693,2020


In [31]:
#exporting to a csv file
csv_data = df.to_csv(index=False)

with open('final_ODI_data.csv', 'w') as csv_file:
    csv_file.write(csv_data)

from google.colab import files
files.download('final_ODI_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>